<a href="https://colab.research.google.com/github/arlianisarumy/Clickbait-Detection-Using-FastText/blob/main/modelling_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fasttext
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 708.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199770 sha256=fbdeba3276c0a47d2b17d11557a38e17386fd735f383723132aada6ce9116628
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import nltk, warnings; warnings.simplefilter('ignore')
import random
import numpy as np
import fasttext
import csv
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import plotly.express as px

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/skripsi_arum/data/data_modelling2.csv')

In [ ]:
df.head()

,title,label,label_score,title_cleaned
0,"Masuk Radar Pilwalkot Medan, Menantu Jokowi Be...",non-clickbait,0,masuk radar pilwalkot medan menantu jokowi tem...
1,Malaysia Sudutkan RI: Isu Kabut Asap hingga In...,non-clickbait,0,malaysia sudut ri isu kabut asap hingga invasi...
2,Viral! Driver Ojol di Bekasi Antar Pesanan Mak...,clickbait,1,viral driver ojol di bekas antar pesan makan p...
3,"Kemensos Salurkan Rp 7,3 M bagi Korban Kerusuh...",non-clickbait,0,kemensos salur rp bagi korban rusuh sosial di ...
4,"Terkait Mayat Bayi Mengenaskan di Tangerang, S...",non-clickbait,0,kait mayat bayi enas di tangerang orang pria t...


In [ ]:
data = df[['title_cleaned','label_score']]
data.head()

,title_cleaned,label_score
0,masuk radar pilwalkot medan menantu jokowi tem...,0
1,malaysia sudut ri isu kabut asap hingga invasi...,0
2,viral driver ojol di bekas antar pesan makan p...,1
3,kemensos salur rp bagi korban rusuh sosial di ...,0
4,kait mayat bayi enas di tangerang orang pria t...,0


In [ ]:
X = data.title_cleaned.values.tolist()
y = data.label_score.values.tolist()

In [ ]:
data_utama, test_X, label_utama, test_y = train_test_split(X, y, test_size=0.20, stratify=y, random_state=42)

In [ ]:
label = ['__label__'+ str(y) for y in label_utama]
label_test = ['__label__'+ str(y) for y in test_y]

In [ ]:
data2=pd.DataFrame({'label_score': label,'text': data_utama})

In [ ]:
label_title = data2['label_score'] + " " + data2['text']

In [ ]:
label_title=label_title.tolist()

In [ ]:
label_title

['__label__0 bursa transfer liga tutup pss sleman datang main lagi',
 '__label__0 diagnosis kembung biasa bocah tahun tinggal akibat kanker perut',
 '__label__0 pertama di indonesia bambu olah jadi energi listrik untuk rang desa',
 '__label__0 caleg perempuan kian kuat di milu',
 '__label__1 singgung soal makna lagu kasih sayang kepada orangtua ini jawab mawang',
 '__label__1 brakk truk ngebut lindas kendara motor di jalan gatsu medan',
 '__label__0 perintah baru terima ganti rugi dari usaha bakar lahan',
 '__label__1 demo surabaya cenderung kondusif duga ini alas',
 '__label__0 icw kritik pidato jokowi soal revisi uu kpk tidak ada kuat',
 '__label__1 mau tahu seperti apa huawei mate pro ini spesifikasi dan gambar',
 '__label__0 nunung jefri nichol tangkap atta halilintar tidak keren dengan narkoba',
 '__label__1 massa gejayan panggil mulai kumpul singgung rezim pinokio',
 '__label__1 main film horor prisia nasution syuting jam',
 '__label__1 hari tiga final gala catur emas penuh kejut

In [ ]:
param_grid = {
    'lr': [0.001, 0.01, 0.1, 1],
    'wordNgrams': [2, 3, 4, 5],
    'epoch': [5, 10, 15, 20]
}
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
best_score = 0
best_params = {}

In [ ]:
results = []
for lr in param_grid['lr']:
    for wordNgrams in param_grid['wordNgrams']:
        for epoch in param_grid['epoch']:
          avg_accuracy = 0
          fold_accuracy_list = []
          for train_idx, val_idx in skf.split(np.zeros(len(label_title)), np.zeros(len(label_title))):
              train_data = [label_title[i] for i in train_idx]
              val_data = [label_title[i] for i in val_idx]

              # Simpan data ke file temporary (FastText membutuhkan file)
              with open('train.txt', 'w') as f:
                  f.write('\n'.join(train_data))
              with open('val.txt', 'w') as f:
                  f.write('\n'.join(val_data))

              # Training dengan FastText
              model = fasttext.train_supervised('train.txt', lr=lr, wordNgrams=wordNgrams, epoch=epoch)

              # Evaluasi model
              result = model.test('val.txt')
              accuracy = result[1]
              avg_accuracy += accuracy
              fold_accuracy_list.append(accuracy)

          avg_accuracy /= k_folds

          results.append({'lr': lr, 'wordNgrams': wordNgrams, 'epoch': epoch,'Accuracy': avg_accuracy})
          print("lr = ", lr, "wordNgrams = ", wordNgrams, "epoch = ", epoch, "akurasi = ", avg_accuracy)
          print("fold =", fold_accuracy_list)
          if avg_accuracy > best_score:
              best_score = avg_accuracy
              best_params = {'lr': lr, 'wordNgrams': wordNgrams, 'epoch': epoch}

print("Best Parameters:", best_params)
print("Best Average Accuracy:", best_score)

# Create DataFrame from results list
results_df = pd.DataFrame(results)

lr =  0.001 wordNgrams =  2 epoch =  5 akurasi =  0.5814999999999999
fold = [0.58875, 0.5904166666666667, 0.5679166666666666, 0.5820833333333333, 0.5783333333333334]
lr =  0.001 wordNgrams =  2 epoch =  10 akurasi =  0.5811666666666666
fold = [0.5891666666666666, 0.5904166666666667, 0.5675, 0.5816666666666667, 0.5770833333333333]
lr =  0.001 wordNgrams =  2 epoch =  15 akurasi =  0.5808333333333333
fold = [0.5891666666666666, 0.59, 0.5675, 0.5808333333333333, 0.5766666666666667]
lr =  0.001 wordNgrams =  2 epoch =  20 akurasi =  0.5805833333333333
fold = [0.58875, 0.59, 0.5670833333333334, 0.5808333333333333, 0.57625]
lr =  0.001 wordNgrams =  3 epoch =  5 akurasi =  0.58175
fold = [0.58875, 0.5925, 0.5675, 0.5833333333333334, 0.5766666666666667]
lr =  0.001 wordNgrams =  3 epoch =  10 akurasi =  0.5814166666666666
fold = [0.5891666666666666, 0.5916666666666667, 0.5675, 0.5820833333333333, 0.5766666666666667]
lr =  0.001 wordNgrams =  3 epoch =  15 akurasi =  0.58125
fold = [0.58875, 0

In [ ]:
results_df.head()

,lr,wordNgrams,epoch,Accuracy
0,0.001,2,5,0.581500
1,0.001,2,10,0.581167
2,0.001,2,15,0.580833
3,0.001,2,20,0.580583
4,0.001,3,5,0.581750


In [ ]:
results_df['lr_log'] = np.log10(results_df['lr'])

In [ ]:
results_df.head()

,lr,wordNgrams,epoch,Accuracy,lr_log
0,0.001,2,5,0.581500,-3.0
1,0.001,2,10,0.581167,-3.0
2,0.001,2,15,0.580833,-3.0
3,0.001,2,20,0.580583,-3.0
4,0.001,3,5,0.581750,-3.0


In [ ]:
# Membuat parallel coordinates plot
fig = px.parallel_coordinates(results_df, color='lr_log', labels={'Accuracy': 'Akurasi', 'lr_log':'lr'},
                              dimensions=['wordNgrams', 'lr_log', 'epoch', 'Accuracy'],
                              color_continuous_scale=px.colors.sequential.Magma)

fig.show()

In [ ]:
y_predict = [model.predict(text)[0][0] for text in test_X]

In [ ]:
confusion_matrix(label_test, y_predict)

array([[1509,  233],
       [ 472,  786]])

In [ ]:
print(classification_report(label_test, y_predict, target_names=['non-clickbait','clickbait']))

               precision    recall  f1-score   support

non-clickbait       0.76      0.87      0.81      1742
    clickbait       0.77      0.62      0.69      1258

     accuracy                           0.77      3000
    macro avg       0.77      0.75      0.75      3000
 weighted avg       0.77      0.77      0.76      3000

